# Pierwsze podejście do tworzenia bazy danych

### Markery

In [1]:
from pyomeca import Markers
data_path = r"Z:\baza\2020-08-05-B0444-S01-E01-T01-K1.c3d"

channels = ["B0444:LASI"]
markers = Markers.from_c3d(data_path, usecols=channels)

In [2]:
marker = markers.meca.to_wide_dataframe()
marker

,ones_B0444:LASI,x_B0444:LASI,y_B0444:LASI,z_B0444:LASI
time,,,,
0.000,1.0,-417.799988,-259.638977,981.658569
0.005,1.0,-417.666168,-259.747437,981.468689
0.010,1.0,-417.533875,-259.838684,981.281860
0.015,1.0,-417.403748,-259.912079,981.100464
0.020,1.0,-417.276306,-259.966949,980.926392
...,...,...,...,...
3.260,1.0,-443.587311,-248.724869,968.667480
3.265,1.0,-443.275818,-248.834518,969.229736
3.270,1.0,-442.978088,-248.934006,969.772827


In [3]:
cols = [c for c in marker.columns if c.lower()[:4] != 'ones']

In [4]:
marker[cols]

,x_B0444:LASI,y_B0444:LASI,z_B0444:LASI
time,,,
0.000,-417.799988,-259.638977,981.658569
0.005,-417.666168,-259.747437,981.468689
0.010,-417.533875,-259.838684,981.281860
0.015,-417.403748,-259.912079,981.100464
0.020,-417.276306,-259.966949,980.926392
...,...,...,...
3.260,-443.587311,-248.724869,968.667480
3.265,-443.275818,-248.834518,969.229736
3.270,-442.978088,-248.934006,969.772827


### EMG

In [5]:
from pyomeca import Analogs

In [6]:
def read_analog_allmuscles(datapath):
    muscles = ["Voltage.1","Voltage.2","Voltage.3","Voltage.4","Voltage.5","Voltage.6","Voltage.7","Voltage.8","Voltage.9","Voltage.10","Voltage.11","Voltage.12","Voltage.13","Voltage.14","Voltage.15","Voltage.16"]
    emg = Analogs.from_c3d(datapath, usecols=muscles)
    return emg

In [7]:
def normalize_emg(emg):
    emg_p = (
    emg.meca.band_pass(order=2, cutoff=[10, 425])
    .meca.center()
    .meca.abs()
    .meca.low_pass(order=4, cutoff=5, freq=emg.rate)
    .meca.normalize(ref=None, scale=1)
    )
    return emg_p 

In [8]:
def emg_full_preproces(datapath):
    emg_data = read_analog_allmuscles(datapath)
    normalised_emg=normalize_emg(emg_data)
    return normalised_emg

In [37]:
data_path = r"Z:\baza\2020-08-05-B0444-S01-E01-T01-K1.c3d"
emg = emg_full_preproces(data_path)

In [38]:
anal = emg.meca.to_wide_dataframe()
anal

channel,Voltage.1,Voltage.10,Voltage.11,Voltage.12,Voltage.13,Voltage.14,Voltage.15,Voltage.16,Voltage.2,Voltage.3,Voltage.4,Voltage.5,Voltage.6,Voltage.7,Voltage.8,Voltage.9
time,,,,,,,,,,,,,,,,
0.000,0.130209,0.006921,0.005150,0.137743,0.010068,0.019596,0.042196,0.032482,-0.002841,0.039229,0.019872,-0.005555,0.004186,0.007467,0.006612,0.038659
0.001,0.134482,0.007142,0.005747,0.139689,0.010137,0.020496,0.045231,0.035132,-0.002437,0.040732,0.021271,-0.004959,0.004688,0.007938,0.005743,0.044993
0.002,0.138768,0.007363,0.006340,0.141630,0.010200,0.021397,0.048265,0.037776,-0.002032,0.042239,0.022666,-0.004362,0.005190,0.008409,0.004878,0.051315
0.003,0.143065,0.007586,0.006931,0.143564,0.010260,0.022296,0.051297,0.040414,-0.001628,0.043748,0.024057,-0.003764,0.005691,0.008877,0.004018,0.057623
0.004,0.147372,0.007809,0.007518,0.145492,0.010316,0.023195,0.054327,0.043045,-0.001224,0.045260,0.025444,-0.003164,0.006190,0.009344,0.003165,0.063915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.280,0.303972,0.013873,0.036002,0.299654,0.031056,0.047301,0.091909,0.172446,0.017332,0.118498,0.122743,0.092152,0.041344,0.048439,0.042135,0.372751
3.281,0.303963,0.013874,0.036003,0.299657,0.031056,0.047302,0.091912,0.172454,0.017331,0.118495,0.122743,0.092149,0.041345,0.048438,0.042136,0.372751
3.282,0.303956,0.013874,0.036004,0.299660,0.031056,0.047303,0.091914,0.172461,0.017331,0.118492,0.122743,0.092147,0.041346,0.048437,0.042137,0.372752


### Łączenie odczytów w Markerów i EMG

In [12]:
import pandas as pd
pd.merge(marker, anal, how='inner')

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

**Problem dotyczacy tego, że mamy inną czestotliwość próbkowania: EMG -> 1000 HZ, markery -> 200 Hz**

In [45]:
anal.index

Float64Index([                 0.0,                0.001,                0.002,
                             0.003,                0.004,                0.005,
                             0.006,                0.007,                0.008,
              0.009000000000000001,
              ...
                             3.275,   3.2760000000000002,                3.277,
                             3.278,                3.279,   3.2800000000000002,
                             3.281,                3.282,                3.283,
                3.2840000000000003],
             dtype='float64', name='time', length=3285)

In [58]:
anal.index % 200 == 0.001

array([False,  True, False, ..., False, False, False])

In [71]:
anal[anal.index == 0]

channel,Voltage.1,Voltage.10,Voltage.11,Voltage.12,Voltage.13,Voltage.14,Voltage.15,Voltage.16,Voltage.2,Voltage.3,Voltage.4,Voltage.5,Voltage.6,Voltage.7,Voltage.8,Voltage.9
time,,,,,,,,,,,,,,,,
0.0,0.130209,0.006921,0.00515,0.137743,0.010068,0.019596,0.042196,0.032482,-0.002841,0.039229,0.019872,-0.005555,0.004186,0.007467,0.006612,0.038659


In [75]:
import numpy as np
df1 = anal[np.arange(len(anal)) % 5 == 1]

In [76]:
df1

channel,Voltage.1,Voltage.10,Voltage.11,Voltage.12,Voltage.13,Voltage.14,Voltage.15,Voltage.16,Voltage.2,Voltage.3,Voltage.4,Voltage.5,Voltage.6,Voltage.7,Voltage.8,Voltage.9
time,,,,,,,,,,,,,,,,
0.001,0.134482,0.007142,0.005747,0.139689,0.010137,0.020496,0.045231,0.035132,-0.002437,0.040732,0.021271,-0.004959,0.004688,0.007938,0.005743,0.044993
0.006,0.156006,0.008257,0.008682,0.149322,0.010416,0.024988,0.060375,0.048283,-0.000419,0.048287,0.028201,-0.001961,0.007186,0.010270,0.001481,0.076440
0.011,0.177624,0.009382,0.011518,0.158699,0.010605,0.029425,0.075363,0.061190,0.001579,0.055836,0.034978,0.001067,0.009642,0.012536,-0.002538,0.107294
0.016,0.199101,0.010505,0.014228,0.167693,0.010727,0.033762,0.090055,0.073743,0.003542,0.063282,0.041547,0.004121,0.012036,0.014707,-0.006154,0.137261
0.021,0.220208,0.011614,0.016791,0.176190,0.010811,0.037951,0.104319,0.085842,0.005455,0.070524,0.047858,0.007199,0.014350,0.016758,-0.009195,0.166063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.261,0.304789,0.013838,0.035943,0.299336,0.031042,0.047219,0.091685,0.171738,0.017363,0.118821,0.122751,0.092428,0.041232,0.048558,0.042054,0.372672
3.266,0.304401,0.013855,0.035970,0.299490,0.031048,0.047258,0.091792,0.172063,0.017348,0.118663,0.122747,0.092292,0.041287,0.048501,0.042091,0.372711
3.271,0.304160,0.013865,0.035988,0.299584,0.031053,0.047282,0.091858,0.172273,0.017338,0.118569,0.122744,0.092212,0.041319,0.048466,0.042115,0.372735
